## Long Short Term Model (LSTM)

### General Motivation

- The general idea is to capture non-linear relationship of the time series model.
- We aim to acheive, deep time series forecasting of the next hour of electricity using our standard time series window
- Since most of the traditional time series model, don't incur the sequential time modelling we use LSTM to take into account the rolling time window approach


### Core implementation

- We use the empirical mode decomposition (EEMD) LSTM for the modelling our time series data
- This approach takes into account that more we add randomness in the data the better we learn in the deep neural network layer
- We also add time distributed dense layer to learn the distribution in the span of window size as well

### Let's implement it......
### Load Data
Add the path where you keep your data files, 

In [1]:
# To reload libraries after we make changes to them in real time without restarting the Kernel
%load_ext autoreload
%autoreload 2

In [2]:
# Add path to the location of our library. 
import sys
sys.path.append('../src/')
import lib

Import some basic libraries, 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Set a random seed for reproducibility. 

In [4]:
np.random.seed(42)

### Load Data

Add the path where you keep your data files, 

In [5]:
data_path = '../data/'

For the model, I will load price OHLC data where the volume is summed. 

In [6]:
resampled_df = pd.read_pickle(data_path+'hourly_resampled_contracts_ohlcsv_weather.pkl')
resampled_df

contractId   qty                                       \
                    contractId  open  high   low close     sum        var   
2020-03-01 11:00:00   11629792   0.5   3.0   0.1   0.2     7.3   1.933667   
2020-03-01 11:00:00   11629866   6.0   6.0   1.0   5.0    28.0   3.466667   
2020-03-01 12:00:00   11629866   0.5  30.0   0.1   2.4  1755.3  11.694814   
2020-03-01 11:00:00   11629920   1.0   6.0   0.2   0.2    20.7   6.846190   
2020-03-01 12:00:00   11629920   1.0  20.0   0.1   3.0  1005.3  11.378282   
...                        ...   ...   ...   ...   ...     ...        ...   
2020-08-25 07:00:00   11877196   1.0  25.0   0.1   1.6  2240.2   8.645283   
2020-08-24 19:00:00   11879762   4.5   4.5   4.5   4.5     4.5  -1.000000   
2020-08-24 20:00:00   11879762  10.5  10.5  10.5  10.5    10.5  -1.000000   
2020-08-24 19:00:00   11879763  11.0  11.0  11.0  11.0    11.0  -1.000000   
2020-08-24 20:00:00   11879813  52.5  52.5  52.5  52.5    52.5  -1.000000   

                        px                                      hour  \
                      open   high    low  close        var             
2020-03-01 11:00:00  -0.99  -0.99  -1.00  -1.00   0.000027  0.000000   
2020-03-01 11:00:00   1.30   1.71   1.30   1.41   0.022627  0.000000   
2020-03-01 12:00:00   1.71  18.00  -5.57  -5.57  38.008268  0.035714   
2020-03-01 11:00:00  18.50  18.50  18.50  18.50   0.000000  0.000000   
2020-03-01 12:00:00  18.11  19.00  16.21  17.20   0.468105  0.035714   
...                    ...    ...    ...    ...        ...       ...   
2020-08-25 07:00:00  35.00  50.55   5.00  38.53   6.497054  0.571429   
2020-08-24 19:00:00  38.95  38.95  38.95  38.95  -1.000000  0.000000   
2020-08-24 20:00:00  38.95  38.95  38.95  38.95  -1.000000  0.035714   
2020-08-24 19:00:00  34.95  34.95  34.95  34.95  -1.000000  0.000000   
2020-08-24 20:00:00  37.98  37.98  37.98  37.98  -1.000000  0.000000   

                     air_temp rel_humidity wind_speed  wind_dir  \
                                                                  
2020-03-01 11:00:00  0.246006     0.646465   0.619048  0.583333   
2020-03-01 11:00:00  0.246006     0.646465   0.619048  0.583333   
2020-03-01 12:00:00  0.255591     0.636364   0.619048  0.527778   
2020-03-01 11:00:00  0.246006     0.646465   0.619048  0.583333   
2020-03-01 12:00:00  0.255591     0.636364   0.619048  0.527778   
...                       ...          ...        ...       ...   
2020-08-25 07:00:00  0.437700     0.969697   0.142857  0.194444   
2020-08-24 19:00:00  0.472843     0.929293   0.238095  0.166667   
2020-08-24 20:00:00  0.476038     0.939394   0.238095  0.194444   
2020-08-24 19:00:00  0.472843     0.929293   0.238095  0.166667   
2020-08-24 20:00:00  0.476038     0.939394   0.238095  0.194444   

                    n_prev_hour_contracts  
                                           
2020-03-01 11:00:00              0.146341  
2020-03-01 11:00:00              0.146341  
2020-03-01 12:00:00              0.146341  
2020-03-01 11:00:00              0.146341  
2020-03-01 12:00:00              0.146341  
...                                   ...  
2020-08-25 07:00:00              0.341463  
2020-08-24 19:00:00              0.634146  
2020-08-24 20:00:00              0.658537  
2020-08-24 19:00:00              0.634146  
2020-08-24 20:00:00              0.658537  

[63451 rows x 18 columns]

That's all for now! 

## Basic API calls

One of the easiest thing to try with the library is to forecast a Random Forest model with default hyperparameters. To begin, first import the library, 

In [10]:
from classes import Forecasting

### Step 1.1: Rolling Windows

The class first calculates rolling windows based on the `WINDOW_SIZE` and adds `features` to the data based on the list provided. The name of the available features can be found in the `lib.create_features()` function. The `ohlc` flag specifies if your data has ohlc values, as compared to data having only close prices (c.f. *data/hourly_resampled_contracts.pkl*). 

<div class="alert alert-block alert-danger">
<b>On feature selection:</b> Only select those features that can be calculated using the dataframe you provide above. Features like 'act_px_open' cannot be calculated as we don't have actual prices in the data we have loaded above. Also, order matters! Give the name of the functions in the same order as they appear in the function. 
</div>

To reduce computation time, we have provided a `recalculate` flag. Suppose you have already calculated rolling windows for a set of features. Since the calculation is computationally expensive, setting the flag to *False* means that it will try to load the previously computed windows, based on the `ohlc` flag and `WINDOW_SIZE`. If you set the `recalculate` flag to False and it cannot find your file based on your params, it will recompute automatically. However, the file name cannot tell you which features it has. It is on you to make sure the pkl file you want to load has the features you want to use for your analysis. **Whenever in doubt, just recalculate.**

In [11]:
WINDOW_SIZE = 5
features = ['t', 'weekday_sin', 'weekday_cos', 'run_hour', 
            'n_prev_hour_contracts', 'hour_sin', 'hour_cos', 
            'air_temp', 'rel_humidity', 'wind_speed', 'wind_dir',
            'holidays', 'qty_open', 'qty_high', 'qty_low', 'qty_close',
            'qty_var' ,'qty_sum', 'px_open','px_high', 'px_low', 'px_var']
ohlc = True
recalculate = False
save_to_pickle = False

### Step 1.2: Data Split

Based on the rolling windows generated above, we now compute a Train-Valid-Test split. The `test_set` flag controls if you want a test set. If False, we only have a Train-Valid split. You might not want a test set in cases where you don't have cross-validation. The first split percentage creates a Train-Test from rolling windows and the second carves out another Train-Test from the Test set of first split. 

In [12]:
test_set = True
split_pct_1 = 0.1
split_pct_2 = 0.5

### Step 1.3 Model Selection

Finally, you specify the `regressor` you want to use. The exact naming convention for a specific regressor is in the docstring of the class. `params` accepts a dictionary of model parameters and the `grid_search` is a flag to use `GridSearchCV`. Presently, we support only Scikit-Learn regressors for grid search. If `grid_search` is True, `params` is expected to be a dictionary of parameters to do grid search on (as should be obvious). 

The model is automatically initialised and fitted. A set of regression metrics are calculated and made available. There is also an option to plot feature-importance for some regressors. 

In [13]:
regressor = 'lstm' 
params = {'lstm_layer':[100,75,64,1], 'batch_size':256, 'dropout': 0.25, 'epochs': 15}
grid_search = False 

### Step 1.4: Call the API

Now that we have our paramters set up, we can make the API call to the Forecasting class. 

In [ ]:
x = Forecasting(resampled_df, window_size=WINDOW_SIZE, features=features, ohlc=ohlc, 
                recalculate=recalculate, test_set=test_set, split_pct_1=split_pct_1,
                split_pct_2=split_pct_2, regressor=regressor, params=params, 
                grid_search=grid_search, save_to_pickle=save_to_pickle)

Fitting model...
Epoch 1/15
157/157 [==============================] - 23s 149ms/step - loss: 10.8155 - mse: 264.8038 - mae: 10.8155 - coeff_determination: 0.0272 - val_loss: 4.4334 - val_mse: 75.2307 - val_mae: 4.4334 - val_coeff_determination: 0.7145
Epoch 2/15
157/157 [==============================] - 19s 122ms/step - loss: 4.4414 - mse: 73.0990 - mae: 4.4414 - coeff_determination: 0.7332 - val_loss: 3.8679 - val_mse: 53.0356 - val_mae: 3.8679 - val_coeff_determination: 0.7950
Epoch 3/15
157/157 [==============================] - 19s 124ms/step - loss: 4.0661 - mse: 58.8584 - mae: 4.0661 - coeff_determination: 0.7837 - val_loss: 3.6000 - val_mse: 46.0438 - val_mae: 3.6000 - val_coeff_determination: 0.8209
Epoch 4/15
157/157 [==============================] - 18s 115ms/step - loss: 3.8497 - mse: 52.6666 - mae: 3.8497 - coeff_determination: 0.8037 - val_loss: 3.4207 - val_mse: 42.7289 - val_mae: 3.4207 - val_coeff_determination: 0.8344
Epoch 5/15
157/157 [============================

In [38]:
x.metrics

,value
MSE,29.115072
RMSE,5.395838
MAE,2.637505
R2,0.883393
